In [29]:
import pandas as pd
import numpy as np

-
- Year: Год выпуска или производства
- Model: Название или код модели автомобиля
- Car/Suv: Тип автомобиля (легковой автомобиль или внедорожник)
- Title: Название или описание автомобиля
- UsedOrNew: Состояние автомобиля (б/у или новый)
- Transmission: Тип трансмиссии (механическая или автоматическая)
- Engine: Объем двигателя или мощность (в литрах или киловаттах)
- DriveType: Тип привода (передний, задний или полный)
- FuelType: Тип топлива (бензин, дизель, гибрид или электричество)
- FuelConsumption: Потребление топлива (в литрах на 100 км)
- Kilometres: Пробег автомобиля (в километрах)
- ColourExtInt: Цвет автомобиля (внешний и внутренний)
- Location: Местонахождение автомобиля (город и штат)
- CylindersinEngine: Количество цилиндров в двигателе
- BodyType: Форма или стиль кузова автомобиля (седан, хэтчбек, купе и т.д.)
- Doors: Количество дверей в автомобиле
- Seats: Количество сидений в автомобиле

In [32]:
DATA_PATH ='../dataset/Australian_Vehicle_Prices.csv'

In [34]:
df = pd.read_csv(DATA_PATH)